In [0]:
!unzip -q "/content/drive/My Drive/data.zip"

In [3]:
!git clone https://github.com/ialhashim/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 235 (delta 0), reused 1 (delta 0), pack-reused 231
Receiving objects: 100% (235/235), 11.80 MiB | 14.63 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [4]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-03 10:07:50--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.96.122
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.96.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  89.7MB/s    in 1.8s    

2020-05-03 10:07:52 (89.7 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [1]:
import os
import glob
import argparse
import matplotlib
import cv2
from PIL import Image
import numpy as np

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from DenseDepth.layers import BilinearUpSampling2D
from DenseDepth.utils import predict, display_images, save_images
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [0]:
def load_images(image_files):
    loaded_images = []
    for file in image_files:
        x = np.clip(np.asarray(Image.open( file ).resize((448, 448), Image.BICUBIC), dtype=float) / 255, 0, 1) 
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

In [4]:
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model('/content/DenseDepth/nyu.h5', custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format('/content/DenseDepth/nyu.h5'))

Loading model...

Model loaded (/content/DenseDepth/nyu.h5).


In [0]:
source = '/content/data/fg_bg/'
destination = '/content/fg_bg_depth/'

os.mkdir(destination)

source_folders = sorted(glob.glob(source+'*'))

for source_folder in source_folders:

  folder_name = source_folder.split('/')[-1]+'/'
  os.mkdir(destination+folder_name)
  print("Processing "+folder_name+"...")

  image_paths = sorted(glob.glob(source_folder+'/*'))

  for i in range(8):
    x = 500*i
    y = 500*(i+1)
    inputs = load_images(image_paths[x:y] )
    outputs = predict(model, inputs)

    for j in range(x, y):
      img_name = image_paths[j].split('/')[-1]
      plt.imsave(destination+folder_name+img_name, outputs[j%500][:,:,0], cmap = 'gray')

Processing bg_001/...


In [0]:
# !rm -r '/content/fg_bg_depth'

In [0]:
!zip -r '/content/drive/My Drive/fg_bg_depth.zip' 'fg_bg_depth'